<a href="https://colab.research.google.com/github/A-Ighalo/Data-Science-Projects/blob/main/Product_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Welcome to the Notebook**

### Task 1 - Set up the project

Installing the needed modules.

In [83]:
!pip install sentence-transformers

Import the modules

In [84]:
#!pip install openai==1.16.2 python-dotenv pyspark

#!pip install openai==1.1.0 httpx==0.27.2 python-dotenv==1.0.0

In [85]:
!pip install python-dotenv

In [119]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType

from pyspark.ml.feature import VectorAssembler, PCA
from pyspark.ml.clustering import KMeans
import plotly.express as px



```
# This is formatted as code
```

Setup the Sentence Transformer API

In [87]:
from sentence_transformers import SentenceTransformer

# Load a free sentence embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")  # No API key required


Create a Spark session

In [88]:
spark =SparkSession.builder.appName("RecommenderSystem").getOrCreate()
spark

Loading the dataset

In [89]:
file_path = "/content/products_dataset.csv"

df = spark.read.csv(file_path, header = True, inferSchema = True , samplingRatio = 1)
df.show()



+----------+--------------------+--------------------+
|product_id|               title|         description|
+----------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|
|        P1|Turmode 30 ft. RP...|If you need more ...|
|        P2|Large Tapestry Bo...|Polyester cover r...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|
|        P9|Traditional Silve...|This transitional...|
|       P10|15 in. x 59 in. O...|Its easy to add a...|
|       P11|1 qt. #350F-7 Wil...|BEHR PREMIUM PLUS...|
|       P12|Anthracite Cordle...|BlindsAvenue ligh...|
|       P13|SlimGrip 78-Inch ...|Luverne SlimGrip ...|
|       P14|6 in. x 28 in. x ...|Our Rustic Collec...|
|       P1

List of 8 products recently viewed by the user.

In [90]:
recently_viewed_products = [
    'P316',
    'P333',
    'P1115',
    'P1691',
    'P1082',
    'P397',
    'P1441',
    'P1054',
]

### Task 2 - Prepare the dataset

Combine `title` and `description` Columns

In [91]:
df = df.withColumn("Combined_text",concat_ws(" ",df.title,df.description))
df.show()

+----------+--------------------+--------------------+--------------------+
|product_id|               title|         description|       Combined_text|
+----------+--------------------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|Men's 3X Large Ca...|
|        P1|Turmode 30 ft. RP...|If you need more ...|Turmode 30 ft. RP...|
|        P2|Large Tapestry Bo...|Polyester cover r...|Large Tapestry Bo...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|16-Gauge-Sinks Ve...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|Men's Crazy Horse...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|Mariana 6 ft. Mul...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|5 gal. #650C-2 Po...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|7/8 in. x 4-1/2 i...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|Ring Gold Bar Car...|
|        P9|Traditional Silve...|This transitional...|Traditional Silve...|
|       P10|

get the combined_text column and convert it into a list

In [92]:
list_combined_text = df.select("combined_text").rdd.flatMap(lambda x: x).collect()
print(list_combined_text[:4])

["Men's 3X Large Carbon Heather Cotton/Polyester Rain Defender Paxton Heavyweight Hooded Zip-Front Sweatshirt This heavyweight, water-repellent hooded sweatshirt has a zip front for fast layering. ORIGINAL FIT. 13 oz., 75% cotton/25% polyester blend with Rain Defender durable water repellent. Attached, jersey-lined three-piece hood with drawcord closure. Antique-finish brass front zipper. Two front hand-warmer pockets have a hidden security pocket inside. Stretchable, spandex-reinforced rib-knit cuffs and waistband. Locker loop facilitates hanging.", "Turmode 30 ft. RP TNC Female to RP TNC Male Adapter Cable If you need more length between your existing wireless device and Hi-Gain Antenna, this is the product for you. It's compatible with most Wi-Fi Antennas, so it is easy for you to extend your wireless network. Just replace your existing cable that runs between your wireless device and Antenna and you're ready to use your network with extended range.", 'Large Tapestry Bolster Bed Pol

Use the sentence transformer text embedding model to create the vector embeddings.

In [93]:
from sentence_transformers import SentenceTransformer

# Load the free sentence embedding model
# Generate embeddings locally using sentence-transformers
# Convert embeddings to list format for consistency
# Print the first two embeddings (for checking)

model = SentenceTransformer("all-MiniLM-L6-v2")  # No need for API key
embedding_vectors = model.encode(list_combined_text)
embedding_vectors = [vec.tolist() for vec in embedding_vectors]
print(embedding_vectors [:2])


[[-0.03837726637721062, 0.08182481676340103, -0.031021270900964737, 0.027743864804506302, 0.1480117291212082, 0.025740714743733406, 0.10736899822950363, -0.06155850738286972, -0.002209707396104932, 0.05575357377529144, 0.00039459491381421685, -0.032364897429943085, 0.047588665038347244, 0.07128600776195526, 0.041558921337127686, -0.01358597818762064, -0.008206640370190144, 0.02266596257686615, -0.0014357659965753555, -0.017309827730059624, -0.03722722828388214, -0.020115384832024574, 0.03947386145591736, 0.04966813325881958, -0.04700634256005287, -0.029382463544607162, -0.006941359955817461, 0.037306517362594604, -0.04411707818508148, -0.06272760033607483, 0.002188641345128417, -0.020608022809028625, 0.044990699738264084, -0.014235887676477432, 0.037747759371995926, 0.004783031065016985, 0.04986855760216713, -0.024538099765777588, -0.044512491673231125, 0.10824702680110931, -0.03172667324542999, 0.02313806489109993, -0.015803921967744827, -0.011280160397291183, -0.04080011323094368, 0.

Let't put the embedding vectors into our original dataframe

Convert embedding vectors list into a Pyspark DataFrame

In [94]:
features_column_names = [f"embedding_{i}" for i in range(len(embedding_vectors[0]))]
df_embeddings = spark.createDataFrame(embedding_vectors, schema=features_column_names)
df_embeddings.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

Add unique `row_id` to each row in the pysaprk dataframe

In [95]:
df_embeddings = df_embeddings.repartition(1).withColumn("row_id", F.monotonically_increasing_id())
df_embeddings.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

Add unique `row_id` to each row in our main pyspark dataframe `df`

In [96]:
df = df.repartition(1).withColumn("row_id",F.monotonically_increasing_id())
df.show()

+----------+--------------------+--------------------+--------------------+------+
|product_id|               title|         description|       Combined_text|row_id|
+----------+--------------------+--------------------+--------------------+------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|Men's 3X Large Ca...|     0|
|        P1|Turmode 30 ft. RP...|If you need more ...|Turmode 30 ft. RP...|     1|
|        P2|Large Tapestry Bo...|Polyester cover r...|Large Tapestry Bo...|     2|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|16-Gauge-Sinks Ve...|     3|
|        P4|Men's Crazy Horse...|This 9 in. black ...|Men's Crazy Horse...|     4|
|        P5|Mariana 6 ft. Mul...|With robust struc...|Mariana 6 ft. Mul...|     5|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|5 gal. #650C-2 Po...|     6|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|7/8 in. x 4-1/2 i...|     7|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|Ring Gold Bar Car...|     8|
|   

Let's join the two dataframes

In [97]:
df = df.join(df_embeddings,on= "row_id",how = "inner").drop("row_id")
df.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

### Task 3 - Cluster products using K-means

Assemble the 512 Embedding Columns into a Single 'features' Column

In [98]:
assembler = VectorAssembler(inputCols = features_column_names,outputCol= "features")
data = assembler.transform(df)
data = data.select(["product_id","title","description","features"])
data.show()

+----------+--------------------+--------------------+--------------------+
|product_id|               title|         description|            features|
+----------+--------------------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|[-0.0383772663772...|
|        P1|Turmode 30 ft. RP...|If you need more ...|[-0.1224228292703...|
|        P2|Large Tapestry Bo...|Polyester cover r...|[-0.0017703186022...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|[-0.0025794846005...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|[-0.0597802437841...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|[-0.0525395274162...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|[-0.0842302292585...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|[-0.1191974803805...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|[0.00208060559816...|
|        P9|Traditional Silve...|This transitional...|[-0.0049049500375...|
|       P10|

Apply K-Means Clustering with 5 Clusters on the `features` Column

In [99]:
#Used KMeans to find 5 groups
kmeans = KMeans(k = 5, featuresCol = "features", predictionCol = "cluster")
model = kmeans.fit(data)
clustered_data = model.transform(data)
clustered_data.show()




+----------+--------------------+--------------------+--------------------+-------+
|product_id|               title|         description|            features|cluster|
+----------+--------------------+--------------------+--------------------+-------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|[-0.0383772663772...|      3|
|        P1|Turmode 30 ft. RP...|If you need more ...|[-0.1224228292703...|      4|
|        P2|Large Tapestry Bo...|Polyester cover r...|[-0.0017703186022...|      0|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|[-0.0025794846005...|      4|
|        P4|Men's Crazy Horse...|This 9 in. black ...|[-0.0597802437841...|      4|
|        P5|Mariana 6 ft. Mul...|With robust struc...|[-0.0525395274162...|      1|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|[-0.0842302292585...|      2|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|[-0.1191974803805...|      4|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|[0.00208060559816...| 

### Task 4 - Visualize the clusters

Let's reduce the dimensionality of our features for visualization purpose

`512 dimensions => 2 dimensions`

In [100]:
#use a scatterplot to find the clusters in the dataset
pca = PCA(k = 2, inputCol = "features", outputCol = "pcaFeatures")
pca_model = pca.fit(clustered_data)
pca_results = pca_model.transform(clustered_data)
pca_results.show()

+----------+--------------------+--------------------+--------------------+-------+--------------------+
|product_id|               title|         description|            features|cluster|         pcaFeatures|
+----------+--------------------+--------------------+--------------------+-------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|[-0.0383772663772...|      3|[0.12740703879622...|
|        P1|Turmode 30 ft. RP...|If you need more ...|[-0.1224228292703...|      4|[-0.0203798803551...|
|        P2|Large Tapestry Bo...|Polyester cover r...|[-0.0017703186022...|      0|[0.14052453726172...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|[-0.0025794846005...|      4|[0.01844509610692...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|[-0.0597802437841...|      4|[-0.1275817924879...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|[-0.0525395274162...|      1|[0.07207475187808...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|[

In [101]:
pca_df = pca_results.select(["product_id","pcaFeatures","cluster"]).toPandas()
pca_df['x'] = pca_df.pcaFeatures.apply(lambda x: x[0])
pca_df['y'] = pca_df.pcaFeatures.apply(lambda x: x[1])
pca_df

,product_id,pcaFeatures,cluster,x,y
0,P0,"[0.12740703879622128, -0.2523959724462838]",3,0.127407,-0.252396
1,P1,"[-0.020379880355162725, 0.05625480408216055]",4,-0.020380,0.056255
2,P2,"[0.14052453726172104, -0.42073465495339996]",0,0.140525,-0.420735
3,P3,"[0.018445096106929425, 0.10625132822865278]",4,0.018445,0.106251
4,P4,"[-0.12758179248791296, -0.16205148593212293]",4,-0.127582,-0.162051
...,...,...,...,...,...
1995,P1995,"[0.15589462303484147, -0.3038291678660307]",1,0.155895,-0.303829
1996,P1996,"[0.1396634199790696, -0.6296858889845355]",0,0.139663,-0.629686
1997,P1997,"[0.04534142615216209, -0.2560111974811752]",1,0.045341,-0.256011
1998,P1998,"[0.6561779931531204, -0.04634203307049945]",2,0.656178,-0.046342


Let's plot the Clusters

In [102]:
def plot_clusters(pca_df, num_clusters=5):
    """
    Plots a 2D visualization of clusters using Plotly Express.

    Parameters:
    - pca_df (DataFrame): A Pandas DataFrame containing columns 'x', 'y', and 'cluster'.
      'x' and 'y' are the 2D PCA components, and 'cluster' indicates the cluster label.
    - num_clusters (int): The number of unique clusters to display.
    - recently_viewed_df (DataFrame, optional): DataFrame with 'x' and 'y' coordinates for recently viewed products.

    This function creates an interactive scatter plot where each point is colored according to its cluster.
    Recently viewed products are marked as black crosses if provided.

    Returns:
    - fig (Figure): The Plotly figure object for the plot.
    """

    # Create the base cluster plot
    fig = px.scatter(
        pca_df,
        x='x',
        y='y',
        opacity=0.6,
        size_max=4,
        color= pca_df.cluster.astype(str),
        title='2D Visualization of Clusters with Recently Viewed Products',
        labels={'x': 'PCA Component 1', 'y': 'PCA Component 2'},
        category_orders={'cluster': list(range(num_clusters))},
        # show the product id in the tooltip
        hover_data={'product_id': True}

    )

    # Update layout to add legend title and adjust plot settings
    fig.update_layout(legend_title_text='Clusters', legend=dict(x=1, y=1), width=600, height=500)

    return fig

fig = plot_clusters(pca_df)
fig.show()

### Task 5 - Highlight recently viewed products

In [103]:
print("The user has recently viewed the following products: ", recently_viewed_products)

The user has recently viewed the following products:  ['P316', 'P333', 'P1115', 'P1691', 'P1082', 'P397', 'P1441', 'P1054']


Let's have a look at the records in our `clustered_data` dataframe related to the recently viewed products.

In [104]:
filtered_data = clustered_data.where(F.col("product_id").isin(recently_viewed_products))
unique_clusters = filtered_data.select("cluster").distinct().rdd.flatMap(lambda x: x).collect()
unique_clusters

[0, 2]

### Task 6 - Recommend products based on recently viewed products

Let's have a look at the recently viewed products titles

In [105]:
filtered_data.select("title").rdd.flatMap(lambda x: x).collect()

["Mystic Fitz Roy Beige 9' 0 x 12' 0 Area Rug",
 'Florida Shag Beige/Multi 3 ft. x 5 ft. Floral Area Rug',
 '1 gal. #M250-3 Apple Turnover Extra Durable Flat Interior Paint & Primer',
 '1 gal. #HDPG60 Misty Emerald Lake Flat Interior Paint and Primer',
 '1 qt. #S220-7 Molasses Extra Durable Flat Interior Paint & Primer',
 'Modern Gray/Multi 9 ft. x 12 ft. Vibrant Abstract Polyester Area Rug',
 '1 qt. #PPU6-06 Honey Locust Eggshell Enamel Low Odor Interior Paint & Primer',
 'Genet Rust/Red-Brown 8 ft. x 11 ft. Abstract Wool Area Rug']

Let's see the distinct clusters of the recenetly viewed products.

In [106]:
print(unique_clusters)

[0, 2]


Let's find the possible products for the recommendation.

In [107]:
possible_recommendations = clustered_data.filter(clustered_data['cluster'].isin(unique_clusters)).filter(~clustered_data['product_id'].isin(recently_viewed_products))

Let's perform a groupby and generate a list of product IDs that can be recommended for each of the clusters.

In [123]:
recommendations = possible_recommendations.groupby('cluster').agg(F.collect_list('product_id').alias('recommendations'))
recommendations_df = recommendations.toPandas()
recommendations_df['random_recommendations'] = recommendations_df.recommendations.apply(lambda x : np.random.choice(x, 5, replace = False).tolist())
recommendations_df.head()

,cluster,recommendations,random_recommendations
0,0,"[P2, P21, P52, P71, P87, P101, P108, P119, P12...","[P187, P831, P1599, P543, P652]"
1,2,"[P6, P11, P16, P18, P24, P26, P30, P33, P40, P...","[P157, P167, P1145, P913, P1114]"


In [124]:
# write a python function to display the recommendations
def display_recommendations(row):
  # find the title of the product in df
  product_ids = row['random_recommendations']
  cluster = row.cluster

  titles = data.filter(data["product_id"].isin(product_ids)).select("title").collect()

  print("\n")
  print("Recommendations for Cluster:", cluster)
  for title in titles:
    print(title[0])



recommendations_df.apply(display_recommendations, axis=1)



Recommendations for Cluster: 0
Rydal Beige/Ivory 8 ft. x 11 ft. Solid Area Rug
Chain Border Custom Size Beige 7.5 in. W x 36 in. H Indoor Carpet Stair Tread Cover Slip Resistant Backing (Set of 7)
Aspen Green/Red 2 ft. x 7 ft. Geometric Runner Rug
Milas Cream 6 ft. x 9 ft. Oriental Polypropylene Area Rug
Chelsea Light Brown 5 ft. x 7 ft. Oval Border Area Rug


Recommendations for Cluster: 2
1 gal. #520C-3 Rapture Blue Satin Enamel Exterior Paint & Primer
5 gal. PPG1024-7 Friar's Brown Flat Interior Latex Paint
5 gal. #PPU25-09 Foggy London Semi-Gloss Enamel Exterior Paint & Primer
5 gal. #740E-1 Dream Catcher Semi-Gloss Enamel Low Odor Interior Paint & Primer
1 qt. #BXC-09 Dark Burgundy Wine Semi-Gloss Enamel Exterior Paint & Primer


,0
0,None
1,None
